# Add Correction filing for All active existing companies

<b> Purpose: Add Corrections filing for all active existing BENs.</b>

This is a one time (python) script to be run at a given date/time.<br>
Set the configuration (client_id, client_secret, url(s)) for a scpecific environment.<br>
Get access token for authorization.


In [1]:
import requests
import os
from datetime import datetime

# token_url, client_id, client_secret, base_url - update based on environment
token_url = os.getenv('ACCOUNT_SVC_AUTH_URL')
client_id = os.getenv('ACCOUNT_SVC_CLIENT_ID')
client_secret = os.getenv('ACCOUNT_SVC_CLIENT_SECRET')
base_url = os.getenv('LEGAL_API_BASE_URL')

header = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = 'grant_type=client_credentials'

res = requests.post(token_url, data, auth=(client_id, client_secret), headers=header)

# Check the status code of the response
if res.status_code == 200:
    print("Access token returned successfully")
    token = res.json()["access_token"]
    # token = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJreXk4aGdkTzVsM3Nwb3k3RnlPWU4wN1ZIWnhYT1hmYUxiV2tuaTZIZHhFIn0.eyJleHAiOjE3MzY0MTkwNDIsImlhdCI6MTczNjQwMTA0MiwiYXV0aF90aW1lIjoxNzM2Mzk5MjYwLCJqdGkiOiIxMDU4YWEzZi1kZWEwLTQxNTAtYTY4Zi1kYjIwMjBlN2YwYTIiLCJpc3MiOiJodHRwczovL2Rldi5sb2dpbnByb3h5Lmdvdi5iYy5jYS9hdXRoL3JlYWxtcy9iY3JlZ2lzdHJ5IiwiYXVkIjpbInNiYy1hdXRoLXdlYiIsImFjY291bnQtc2VydmljZXMiLCJjb21tb24tc2VydmljZXMiLCJlbnRpdHktc2VydmljZXMiLCJOYW1lWC1EZXYiLCJhY2NvdW50Il0sInN1YiI6IjFjZDY4NTllLTlkMDAtNDI0YS1iZWRlLTY3NTc5NzI5YzIyNCIsInR5cCI6IkJlYXJlciIsImF6cCI6ImVudGl0eS13ZWIiLCJub25jZSI6Ijg1NDI0MTY5LWNjNDEtNGQwYS1hZDBlLWYxMzdiMzJiYzFhZSIsInNlc3Npb25fc3RhdGUiOiI0MjllNGQxZS1hMTg5LTRhOTMtYjJiZS1hMDJhNGY2MGJhYjQiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly9idXNpbmVzcy1maWxpbmdzLWRldi5hcHBzLnNpbHZlci5kZXZvcHMuZ292LmJjLmNhIiwiaHR0cHM6Ly9idXNpbmVzcy1jcmVhdGUtZGV2LmFwcHMuc2lsdmVyLmRldm9wcy5nb3YuYmMuY2EiLCJodHRwOi8vbG9jYWxob3N0OjgwODAiLCJodHRwczovL2Rldi5iY3JlZ2lzdHJ5LmNhIiwiaHR0cDovL2xvY2FsaG9zdDo4MDgxIiwiKiIsImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHBzOi8vYnVzaW5lc3MtZWRpdC1kZXYuYXBwcy5zaWx2ZXIuZGV2b3BzLmdvdi5iYy5jYSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsibWFuYWdlX2J1c2luZXNzIiwibmFtZXNfYXBwcm92ZXIiLCJuYW1lc192aWV3ZXIiLCJmYXNfZWRpdCIsImZhc192aWV3IiwibWFuYWdlX2FjY291bnRzIiwic2VhcmNoIiwidmlldyIsImNyZWF0ZV9jcmVkaXRzIiwib2ZmbGluZV9hY2Nlc3MiLCJzdXNwZW5kX2FjY291bnRzIiwidW1hX2F1dGhvcml6YXRpb24iLCJtYWtlX3BheW1lbnQiLCJkZWZhdWx0LXJvbGVzLWJjcmVnaXN0cnkiLCJmYXNfbGluayIsIm5hbWVzX2VkaXRvciIsInZpZXdfYWNjb3VudHMiLCJtaHJfcGF5bWVudCIsImVkaXQiLCJmYXNfcmVwb3J0cyIsImZhc190cmFuc2FjdGlvbiIsInN0YWZmIiwiZmFzX3NlYXJjaCIsImZhc19jcmVhdGUiLCJmYXNfcmVmdW5kIiwiZmFzX3VzZXIiLCJmYXNfcmVmdW5kX2FwcHJvdmVyIiwicHByIiwibWhyIiwiZmFzX3ZvaWQiLCJuYW1lc19tYW5hZ2VyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsInNpZCI6IjQyOWU0ZDFlLWExODktNGE5My1iMmJlLWEwMmE0ZjYwYmFiNCIsImZpcnN0bmFtZSI6IktldGFraSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwicm9sZXMiOlsibWFuYWdlX2J1c2luZXNzIiwibmFtZXNfYXBwcm92ZXIiLCJuYW1lc192aWV3ZXIiLCJmYXNfZWRpdCIsImZhc192aWV3IiwibWFuYWdlX2FjY291bnRzIiwic2VhcmNoIiwidmlldyIsImNyZWF0ZV9jcmVkaXRzIiwib2ZmbGluZV9hY2Nlc3MiLCJzdXNwZW5kX2FjY291bnRzIiwidW1hX2F1dGhvcml6YXRpb24iLCJtYWtlX3BheW1lbnQiLCJkZWZhdWx0LXJvbGVzLWJjcmVnaXN0cnkiLCJmYXNfbGluayIsIm5hbWVzX2VkaXRvciIsInZpZXdfYWNjb3VudHMiLCJtaHJfcGF5bWVudCIsImVkaXQiLCJmYXNfcmVwb3J0cyIsImZhc190cmFuc2FjdGlvbiIsInN0YWZmIiwiZmFzX3NlYXJjaCIsImZhc19jcmVhdGUiLCJmYXNfcmVmdW5kIiwiZmFzX3VzZXIiLCJmYXNfcmVmdW5kX2FwcHJvdmVyIiwicHByIiwibWhyIiwiZmFzX3ZvaWQiLCJuYW1lc19tYW5hZ2VyIl0sIm5hbWUiOiJEZW9kaGFyLCBLZXRha2kgQ0lUWjpFWCIsImlkcF91c2VyaWQiOiIzM0Q1QzU1MkQwNTU0MTQ3OTFEQTQ5QTkyMkNGOUQxMyIsInByZWZlcnJlZF91c2VybmFtZSI6ImtkZW9kaGFyQGlkaXIiLCJnaXZlbl9uYW1lIjoiS2V0YWtpIiwiZmFtaWx5X25hbWUiOiJEZW9kaGFyIiwibG9naW5Tb3VyY2UiOiJJRElSIiwiZW1haWwiOiJrZXRha2kuZGVvZGhhckBnb3YuYmMuY2EiLCJsYXN0bmFtZSI6IkRlb2RoYXIiLCJ1c2VybmFtZSI6ImtkZW9kaGFyQGlkaXIifQ.UQcaIGtrm37glD3khEivzxzp_uG2eApO8GHXeXZk1auAfooKPItKdotbJGNcHivmRLPpClFtdz-BEZyztstpXLAYE4T21ONDqi5bpYeDpmC0XB--UukVLCNK_KNgKRsHbt3KqpH3VTrtlomkN_D5wV6O9waVjR233IQ6QWiskgWwyrLM7LDdftkkUTRwnAmLdDgIrkKbpVx3fGAGHcKk3zqFnkR1c558QL9lxAeKgC0ZObLffTFl3pWDDlDVgwO3lCa7s0hvJsagX1pdygxadvX45txtC6eUv5Mr0TLxadoOGLHIOpJfLtJDxz2kOrBSw-t_MOKeVDMKPtez9kzx_A'
else:
    print(f"Failed to make POST request. Status code: {res.status_code}")
    print(res.text)  # Print the error message if the request fails



Access token returned successfully


Call API (POST) endpoint to createCorrection filing with details as Ben correction statement for businesses.

In [2]:
from urllib.parse import urljoin
from corrections_output import correction_businesses

current_date = datetime.now().date().isoformat()
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

# loop through list of businesses to create filing
for correction_businesse in correction_businesses:
    identifier = correction_businesse[0]
    filind_id = correction_businesse[1]
    correction_filing_data = {
        "filing": {
            "header": {
                "name": "correction",
                "date": current_date,
                "certifiedBy": "system"
            },
            "business": {
                "identifier": identifier,
                "legalType": "BC"
            },
            "correction": {
                "details": "First correction",
                "correctedFilingId": filind_id,
                "correctedFilingType": "incorporationApplication",
                "comment": f"""Correction for Incorporation Application filed on {current_date} \n
                            BC benefit company statement contained in notice of articles as required under section 
                            51.992 of the Business Corporations Act corrected from “This company is a benefit company 
                            and, as such, has purposes that include conducting its business in a responsible and 
                            sustainable manner and promoting one or more public benefits” to 
                            “This company is a benefit company and, as such, is committed to conducting its business in 
                            a responsible and sustainable manner and promoting one or more public benefits”."""
            }
        }
    }

    filing_url = urljoin(base_url, f"/api/v2/businesses/{identifier}/filings")
    rv = requests.post(filing_url, headers=headers, json=correction_filing_data)

    # Check the status code of the response
    if rv.status_code == 201:
        print(f"Correction created successfully for {identifier}")
    else:
        print(f"Failed to make POST request. Status code: {rv.status_code}")
        print(rv.text)  # Print the error message if the request fails
        


Correction created successfully for BC0871147
Correction created successfully for BC0871183
Correction created successfully for BC0871186
